
Similaridade com GENSIM
======================

O notebook em questão demonstra como buscar similaridade por exemplo de uma texto em um conjunto de documentos.
Utilizaremos para isto a biblioteca Python denominada Gensim. Esta biblioteca é utilizadano processamento de linguagem natural (NLP), em funções como indexação de documentos e análise de similaridade entre textos.

Os principais conceitos utilizados em gensim são:

    Document(o): algum texto.

    Corpus: uma coleção de documentos.

    Vector: uma representação matemática de um documento.

    Model(o): an algorithm for transforming vectors from one representation to another.

A seguir o passo a passo para efetuarmos análise de Simularidade pelo Gensim.


Criando o Corpus
-------------------

Antes de qualquer coisa precisamos criar um Corpus.
Nos documentos originais existem palavras que não contribuem para a análise semântica, 
por exemplo, artigos e preposições. Estas palavras serão removidas do nosso corpus. 
Além disso realizaremos o processo de "steeming", ou seja, o processo de reduzir
palavras flexionadas (ou às vezes derivadas) ao seu tronco (stem) base ou raiz, 
geralmente uma forma da palavra escrita. Ex: Devolver e Devolverei se tranformam em devolv
Além do stemming faremos o processo de tokenização, processo de converter uma sequência de 
caracteres em uma sequência de tokens (no nosso caso palavras).




In [1]:
from collections import defaultdict
from gensim import corpora
#from nltk.stem import PorterStemmer /*Stemmer para Inglês*/
from nltk.stem import RSLPStemmer

#Comando para importar o Stemmer e stopwords do idioma Português
#import nltk
#nltk.download('rslp') 
#nltk.download('stopwords')

ps = RSLPStemmer() #Portuguese Stemmer
# Esta é nossa lista de documentos original,
# pode ser por exemplo todas as perguntas que podem ser feitas em um chatbot
documentos = [
    "Qual o status do meu pedido",
    "Meu pedido está em que status",
    "Como acionar um vendedor",
    "Qual vendedor pode me atender",
    "Como devolver o produto",
    "Devolverei um produto e pedido",
    "Quero fazer uma devolução de produto",
    "Tenho uma reclamação",
    "Quero fazer uma reclamação",
    "em que dia será entregue meu produto",
    "Qual o prazo de entrega",
    "Qual a data de entrega",
    "Qual o catálago de produtos",
    "Como acesso o catálago de produtos",
    "Quero devolver o pedido"]


stoplist = set('o a em de do da uma um meu minha me te e que'.split())
#Criamos nossa própria lista de stopwords, mas se quisesse utilizar todas as stopwords do portugues poderia
#ter feito stoplist = nltk.corpus.stopwords.words("portuguese")
texts = [
    [ps.stem(word) for word in documento.lower().split() if word not in stoplist]
    for documento in documentos]
print(texts)


[['qual', 'statu', 'ped'], ['ped', 'est', 'statu'], ['com', 'acion', 'vend'], ['qual', 'vend', 'pod', 'atend'], ['com', 'devolv', 'produt'], ['devolv', 'produt', 'ped'], ['quer', 'faz', 'devoluç', 'produt'], ['tenh', 'reclam'], ['quer', 'faz', 'reclam'], ['dia', 'ser', 'entreg', 'produt'], ['qual', 'praz', 'entreg'], ['qual', 'dat', 'entreg'], ['qual', 'catálag', 'produt'], ['com', 'acess', 'catálag', 'produt'], ['quer', 'devolv', 'ped']]


In [2]:
# removeremos as palavras que aparecem apenas uma vez
frequency = defaultdict(int) 
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]


In [3]:
print('Nosso dicionário possui apenas 12 palavras, 12 tokens únicos, abaixo listados com seus respectivos códigos:')
print (dictionary.token2id)
print ('Nosso corpus reflete nossos documentos, conforme este dicionário, em um formato de vetor, onde para cada documento se tem o código do token e o número de vezes que ele ocorre no corpus como um todo.')
print(corpus)

Nosso dicionário possui apenas 12 palavras, 12 tokens únicos, abaixo listados com seus respectivos códigos:
{'ped': 0, 'qual': 1, 'statu': 2, 'com': 3, 'vend': 4, 'devolv': 5, 'produt': 6, 'faz': 7, 'quer': 8, 'reclam': 9, 'entreg': 10, 'catálag': 11}
Nosso corpus reflete nossos documentos, conforme este dicionário, em um formato de vetor, onde para cada documento se tem o código do token e o número de vezes que ele ocorre no corpus como um todo.
[[(0, 1), (1, 1), (2, 1)], [(0, 1), (2, 1)], [(3, 1), (4, 1)], [(1, 1), (4, 1)], [(3, 1), (5, 1), (6, 1)], [(0, 1), (5, 1), (6, 1)], [(6, 1), (7, 1), (8, 1)], [(9, 1)], [(7, 1), (8, 1), (9, 1)], [(6, 1), (10, 1)], [(1, 1), (10, 1)], [(1, 1), (10, 1)], [(1, 1), (6, 1), (11, 1)], [(3, 1), (6, 1), (11, 1)], [(0, 1), (5, 1), (8, 1)]]


Criando o modelo de Similaridade LSI
--------------------

LSI é um algorítimo de NLP que relaciona de forma semântica um conjunto de documentos e os termos neles contidos. Esta técnica assume que palavras que são próximas em significado vão ocorrer de forma similar em diferentes trechos do texto.
Portanto cria-se uma matriz (vetor) contendo o número de vezes que a palavra (token) apareceu no documento. Documentos são comparados utilizando o cosseno do ângulo dos dois vetores, valores próximo a 1 representam alta similaridaden próximos de zero baixa similaridade.
Abaixo criaremos nosso modelo LSI tendo como base nosso corpus gerado, com seu respectivo dicionário, para geração do vetor LSI espacial com 10 dimensões, que são o número de topicos que passamos como parâmetro.

In [4]:
from gensim import models 
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=10)

In [5]:
#Temos nosso modelo LSI gerado, suponhamos que temos um novo documento, no caso a string doc e queremos verificar como 
#esta string se relaciona como nosso Corpus anterior. Quais documentos são semelhantes semanticamente? Para responder isto utilizaremos
#o modelo LSI gerado
docnovo = "Devolver produto" #novo documento
doc = [ps.stem(word) for word in docnovo.split()] #fazemos o stemming deste novo documento bem como sua tokenização
vec_bow = dictionary.doc2bow(doc)    
vec_lsi = lsi[vec_bow]  # converte a query para um vector LSI
print(doc)
print(vec_bow)
print(vec_lsi)

['devolv', 'produt']
[(5, 1), (6, 1)]
[(0, -0.9882633035409889), (1, -0.4602645609857196), (2, -0.13891911633974807), (3, 0.30152438285289773), (4, 0.3855089709829175), (5, -0.30604202150079923), (6, 0.2593872054695901), (7, 0.15575861299784527), (8, -0.412755613467044), (9, 0.2615849681569986)]


In [6]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi[corpus])  # transforma o corpus original para um vetor espacial LSI e indexa o mesmo
sims = index[vec_lsi]  # executa a query de similaridade do doc novo contra o corpus original
print(list(enumerate(sims)))  # imprime (numero documento corpus original, valor de similaridade) 

[(0, 0.009093601), (1, -0.018258948), (2, 0.009364074), (3, -0.011369765), (4, 0.8320902), (5, 0.86795425), (6, 0.4280177), (7, 0.0019537657), (8, -0.0015561131), (9, 0.5057455), (10, 0.0061714984), (11, 0.0061714984), (12, 0.4133898), (13, 0.43031746), (14, 0.41526675)]


In [7]:
#Verificando e imprimindo em ordem descendente o id do documento original, o valor de similaridade e o texto original
#do documento. Uma visão que mais se relaciona para o que menos se relaciona ao nosso novo documento.
def take_second(elem):
    return elem[1]

sims2 = sorted(list(enumerate(sims)),key=take_second,reverse=True)
#print(sims2)
print('"',docnovo,'" se relaciona de mais para menos com os seguintes documentos:')

j = (list(enumerate(sims2))[-1][0])  #número total de tuplas
i = 0
while i <= j:
    t = sims2[i][0] #indice do documento original
    print(t,sims2[i][1],documentos[t])
    i = i+1

" Devolver produto " se relaciona de mais para menos com os seguintes documentos:
5 0.86795425 Devolverei um produto e pedido
4 0.8320902 Como devolver o produto
9 0.5057455 em que dia será entregue meu produto
13 0.43031746 Como acesso o catálago de produtos
6 0.4280177 Quero fazer uma devolução de produto
14 0.41526675 Quero devolver o pedido
12 0.4133898 Qual o catálago de produtos
2 0.009364074 Como acionar um vendedor
0 0.009093601 Qual o status do meu pedido
10 0.0061714984 Qual o prazo de entrega
11 0.0061714984 Qual a data de entrega
7 0.0019537657 Tenho uma reclamação
8 -0.0015561131 Quero fazer uma reclamação
3 -0.011369765 Qual vendedor pode me atender
1 -0.018258948 Meu pedido está em que status


Referências
https://en.wikipedia.org/wiki/Latent_semantic_analysis#Latent_semantic_indexing
http://www.nltk.org/howto/portuguese_en.html
https://radimrehurek.com/gensim/auto_examples/core/run_similarity_queries.html#sphx-glr-auto-examples-core-run-similarity-queries-py